# 1 Подготовка

## Импорт библиотек

In [ ]:
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import pickle
import requests
import zipfile

## Устанавливаем стиль визуализации

In [2]:
sns.set_theme(style="whitegrid")

# 2 Загрузка данных

In [3]:
# Загрузка данных локально
file_path = '../data/tickets.csv'

data = pd.read_csv(file_path, delimiter=';')

In [ ]:
# Загрузка данных из внешнего хранилища

url = "https://www.kaggle.com/api/v1/datasets/download/adisongoh/it-service-ticket-classification-dataset"  # Адрес для загрузки данных

extract_file = "all_tickets_processed_improved_v3.csv"  # Имя файла в архиве

# Скачивание архива
response = requests.get(url, stream=True)
if response.status_code == 200:
    temp_archive = "../data/temp_archive.zip"
    with open(temp_archive, 'wb') as f:
        f.write(response.content)
    print(f"Данные успешно загружены с {url}")
else:
    raise Exception(f"Ошибка при загрузке данных: {response.status_code} {response.reason}")

# Распаковка архива
extract_dir = '../data'
with zipfile.ZipFile(temp_archive, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("Архив успешно распакован.")
extracted_path = os.path.join(extract_dir, extract_file)

# Очистка временных файлов
os.remove(temp_archive)
print("Временный архив удалён.")

# Загрузка данных в DataFrame
data = pd.read_csv(extracted_path, delimiter=',')

# Переименование колонок при необходимости
rename_mapping = {
    'Document': 'Description',
    'Topic_group': 'Type'
}
data.rename(columns=rename_mapping, inplace=True)

In [ ]:
# Проверка корректной загрузки данных
print("Пример данных:")
display(data.head())

# 3 Исследовательский анализ данных

In [ ]:
# Распределение категорий тикетов
plt.figure(figsize=(10, 6))
sns.countplot(
    y=data['Type'], 
    order=data['Type'].value_counts().index, 
    palette="viridis", 
    legend=False
)
plt.title('Распределение категорий тикетов')
plt.xlabel('Количество')
plt.ylabel('Категория')
plt.show()

In [ ]:
# Длина описаний тикетов
data['description_length'] = data['Description'].apply(len)
plt.figure(figsize=(10, 6))
sns.histplot(data['description_length'], kde=True, bins=30, color='blue')
plt.title('Распределение длины описаний тикетов')
plt.xlabel('Длина описания')
plt.ylabel('Количество')
plt.show()

In [ ]:
# Анализ и визуализация наиболее частых N-грамм

def plot_top_ngrams(corpus, ngram_range=(1, 1), top_n=20, title="Топ N-грамм"):
    vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words='english')
    X = vectorizer.fit_transform(corpus)
    ngram_counts = Counter(dict(zip(vectorizer.get_feature_names_out(), X.sum(axis=0).A1)))
    
    top_ngrams = ngram_counts.most_common(top_n)
    ngrams, counts = zip(*top_ngrams)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(x=counts, y=ngrams, palette='viridis')
    plt.title(title)
    plt.xlabel('Частота')
    plt.ylabel('N-граммы')
    plt.show()

# Топ униграмм
print("Топ униграмм:")
plot_top_ngrams(data['Description'], ngram_range=(1, 1), top_n=20, title="Топ 20 униграмм")

# Топ биграмм
print("Топ биграмм:")
plot_top_ngrams(data['Description'], ngram_range=(2, 2), top_n=20, title="Топ 20 биграмм")

# Топ триграмм
print("Топ триграмм:")
plot_top_ngrams(data['Description'], ngram_range=(3, 3), top_n=20, title="Топ 20 триграмм")

# 4 Предобработка данных

In [8]:
# Удаляем строки с пропущенными значениями
data = data.dropna(subset=['Description', 'Type'])

In [9]:
# Приведение текста к нижнему регистру
data['Description'] = data['Description'].str.lower()

In [10]:
# Удаление лишних символов (шум)
data['Description'] = data['Description'].str.replace(r'[^a-zA-Z\s]', '', regex=True)

In [ ]:
# Выводим несколько обработанных записей
print("\nОбработанные данные:")
display(data.head())

# 5 Разделение на признаки и целевые переменные, векторизация данных TF-IDF

In [12]:
X = data['Description']  # Признаки
y = data['Type']         # Целевая переменная

## Создание обучающей и валидационной выборки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nКоличество записей в обучающей выборке: {len(X_train)}")
print(f"Количество записей в валидационной выборке: {len(X_test)}")

In [ ]:
# Векторизация текста
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1, 2))

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("\nРазмерность векторизованных данных:")
print(f"Обучающие данные: {X_train_tfidf.shape}")
print(f"Валидационные данные: {X_test_tfidf.shape}")

# 6 Обучение модели

### Логистическая регрессия

In [ ]:
lr_model = LogisticRegression(random_state=42, max_iter=1000)

lr_model.fit(X_train_tfidf, y_train)

### Метод K-ближайших соседей (KNN)

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_tfidf, y_train)

### Метод случайного леса (Random Forest)

In [ ]:
rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_model.fit(X_train_tfidf, y_train)

### Метод опорных векторов (SVM)

In [ ]:
svm_model = SVC(random_state=42)
svm_model.fit(X_train_tfidf, y_train)

### Градиентный бустинг (Gradient Boosting)

In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_tfidf, y_train)

# 7 Проверка работоспособности и оценка моделей

In [20]:
# Словарь для сохранения результатов
results = {}

results["Logistic Regression"] = {
    "accuracy": None,
    "classification_report": None
}

results["KNN"] = {
    "accuracy": None,
    "classification_report": None
}

results["Random Forest"] = {
    "accuracy": None,
    "classification_report": None
}

results["SVM"] = {
    "accuracy": None,
    "classification_report": None
}

results["Gradient Boosting"] = {
    "accuracy": None,
    "classification_report": None
}

In [ ]:
# Оценка логистической регрессии
lr_y_pred = lr_model.predict(X_test_tfidf)

print("\nОценка модели: Логистическая регрессия")
lr_accuracy = accuracy_score(y_test, lr_y_pred)
print(f"Accuracy для логистической регрессии: {lr_accuracy:.4f}")
lr_classification_rep = classification_report(y_test, lr_y_pred)
print("Отчёт по классификации для логистической регрессии:")
print(lr_classification_rep)
results["Logistic Regression"] = {
    "accuracy": lr_accuracy,
    "classification_report": lr_classification_rep
}

In [ ]:
# Оценка K-ближайших соседей (KNN)
knn_y_pred = knn_model.predict(X_test_tfidf)

print("\nОценка модели: K-ближайшие соседи (KNN)")
knn_accuracy = accuracy_score(y_test, knn_y_pred)
print(f"Accuracy для KNN: {knn_accuracy:.4f}")
knn_classification_rep = classification_report(y_test, knn_y_pred)
print("Отчёт по классификации для KNN:")
print(knn_classification_rep)
results["KNN"] = {
    "accuracy": knn_accuracy,
    "classification_report": knn_classification_rep
}

In [ ]:
# Оценка случайного леса (Random Forest)
rf_y_pred = rf_model.predict(X_test_tfidf)

print("\nОценка модели: Случайный лес (Random Forest)")
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print(f"Accuracy для Random Forest: {rf_accuracy:.4f}")
rf_classification_rep = classification_report(y_test, rf_y_pred)
print("Отчёт по классификации для Random Forest:")
print(rf_classification_rep)
results["Random Forest"] = {
    "accuracy": rf_accuracy,
    "classification_report": rf_classification_rep
}

In [ ]:
# Оценка метода опорных векторов (SVM)
svm_y_pred = svm_model.predict(X_test_tfidf)

print("\nОценка модели: Метод опорных векторов (SVM)")
svm_accuracy = accuracy_score(y_test, svm_y_pred)
print(f"Accuracy для SVM: {svm_accuracy:.4f}")
svm_classification_rep = classification_report(y_test, svm_y_pred)
print("Отчёт по классификации для SVM:")
print(svm_classification_rep)
results["SVM"] = {
    "accuracy": svm_accuracy,
    "classification_report": svm_classification_rep
}

In [ ]:
# Оценка градиентного бустинга (Gradient Boosting)
gb_y_pred = gb_model.predict(X_test_tfidf)

print("\nОценка модели: Градиентный бустинг (Gradient Boosting)")
gb_accuracy = accuracy_score(y_test, gb_y_pred)
print(f"Accuracy для Gradient Boosting: {gb_accuracy:.4f}")
gb_classification_rep = classification_report(y_test, gb_y_pred)
print("Отчёт по классификации для Gradient Boosting:")
print(gb_classification_rep)
results["Gradient Boosting"] = {
    "accuracy": gb_accuracy,
    "classification_report": gb_classification_rep
}

In [ ]:
# Визуализация сравнительных результатов
print("\nСравнение результатов моделей:")
model_names = list(results.keys())
accuracies = [results[model]["accuracy"] for model in model_names]

# Построение графика точности
plt.figure(figsize=(12, 8))
sns.barplot(x=model_names, y=accuracies, palette='viridis')
plt.title('Сравнение точности различных моделей', fontsize=16)
plt.xlabel('Метод', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

# Шаг 8: Печать всех отчётов классификации
print("\nПодробные отчёты по классификации для каждой модели:\n")
for model, metrics in results.items():
    print(f"\nМодель: {model}")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print("Отчет по классификации:")
    print(metrics["classification_report"])

# 8 Сохранение модели

In [ ]:
# Нахождение модель с наилучшей точностью
best_model_name = max(results, key=lambda x: results[x]['accuracy'])
best_model_accuracy = results[best_model_name]['accuracy']

print(f"Лучшей моделью оказалась: {best_model_name} с точностью {best_model_accuracy:.4f}")

# Сопоставление названия модели с её объектом
model_mapping = {
    "Логистическая регрессия": lr_model,
    "Метод K-ближайших соседей": knn_model,
    "Случайный лес": rf_model,
    "Метод опорных векторов": svm_model,
    "Градиентный бустинг": gb_model
}

In [ ]:
# Сохранение лучшей модели
best_model = model_mapping[best_model_name]
model_path = f"../models/{best_model_name.replace(' ', '_').lower()}_ticket_classifier_model.pkl"

with open(model_path, 'wb') as model_file:
    pickle.dump(best_model, model_file)

# Сохранение векторизатора
vectorizer_path = f"../models/{best_model_name.replace(' ', '_').lower()}_tfidf_vectorizer.pkl"

with open(vectorizer_path, 'wb') as vectorizer_file:
    pickle.dump(vectorizer, vectorizer_file)

print(f"Модель '{best_model_name}' успешно сохранена по пути: {model_path}")
print(f"Векторизатор сохранён по пути: {vectorizer_path}")

# 9 Загрузка и проверка модели

In [ ]:
# Загрузка модели
with open(model_path, 'rb') as model_file:
    loaded_model = pickle.load(model_file)

# Загрузка векторизатора
with open(vectorizer_path, 'rb') as vectorizer_file:
    loaded_vectorizer = pickle.load(vectorizer_file)

print("\nМодель и векторизатор успешно загружены.")

In [ ]:
# Тестирование загруженной модели на новом тексте
new_ticket = ["Reports fail to load when certain filters are applied"]
new_ticket_vectorized = loaded_vectorizer.transform(new_ticket)
new_ticket_prediction = loaded_model.predict(new_ticket_vectorized)

print("\nТестовое описание тикета:", new_ticket[0])
print("Предсказанная категория:", new_ticket_prediction[0])